In [1]:
from transformers import pipeline

# 감성 분석 파이프라인 로드
classifier = pipeline("sentiment-analysis")

# 테스트 텍스트 입력
result = classifier("I love Hugging Face!")
print(result)
# 출력: [{'label': 'POSITIVE', 'score': 0.99}]


C:\Users\jaeyung\AppData\Roaming\Python\Python311\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\jaeyung\AppData\Roaming\Python\Python311\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit 

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

C:\Users\jaeyung\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jaeyung\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998641014099121}]


In [28]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
import torch.nn.utils.prune as prune

# 1. 사전 학습된 모델과 토크나이저 로드
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. 모델의 가중치 정보 확인
print("Pruning 전 모델 구조:")
for name, module in model.named_modules():
    print(f"{name}: {module}")

# 3. Pruning 적용 (가중치의 50%를 제거)
# distilbert.transformer.layer.*.attention.q_lin 등 중요한 레이어를 지정
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):  # Linear Layer에만 Pruning 적용
        prune.l1_unstructured(module, name="weight", amount=0.5)  # 50% 제거

# 4. Pruning된 모델의 가중치 확인
print("\nPruning 후 모델 구조:")
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(f"{name}.weight_pruned: {module.weight}")

# 5. 모델 테스트
# Pruning 후에도 정상 동작하는지 테스트
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 입력 문장 테스트
test_sentence = "I love Hugging Face!"
result = classifier(test_sentence)
print("\n테스트 결과:", result)

# 6. 모델 저장
pruned_model_path = "./pruned_sentiment_model"
model.save_pretrained(pruned_model_path)
tokenizer.save_pretrained(pruned_model_path)

print("\nPruned 모델이 저장되었습니다:", pruned_model_path)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

C:\Users\jaeyung\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jaeyung\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Pruning 전 모델 구조:
: DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.

In [29]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# 저장된 모델 경로
pruned_model_path = "./pruned_sentiment_model"

# 1. 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained(pruned_model_path)
tokenizer = AutoTokenizer.from_pretrained(pruned_model_path)

# 2. 파이프라인 생성
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 3. 테스트 입력
test_sentences = [
    "I love using Hugging Face models!",
    "This is a terrible product.",
    "The movie was okay, not great but not bad either."
]

# 4. 결과 출력
print("\nPruned 모델 테스트 결과:")
for sentence in test_sentences:
    result = classifier(sentence)
    print(f"입력 문장: {sentence}")
    print(f"결과: {result}\n")


Some weights of the model checkpoint at ./pruned_sentiment_model were not used when initializing DistilBertForSequenceClassification: ['classifier.weight_mask', 'classifier.weight_orig', 'distilbert.transformer.layer.0.attention.k_lin.weight_mask', 'distilbert.transformer.layer.0.attention.k_lin.weight_orig', 'distilbert.transformer.layer.0.attention.out_lin.weight_mask', 'distilbert.transformer.layer.0.attention.out_lin.weight_orig', 'distilbert.transformer.layer.0.attention.q_lin.weight_mask', 'distilbert.transformer.layer.0.attention.q_lin.weight_orig', 'distilbert.transformer.layer.0.attention.v_lin.weight_mask', 'distilbert.transformer.layer.0.attention.v_lin.weight_orig', 'distilbert.transformer.layer.0.ffn.lin1.weight_mask', 'distilbert.transformer.layer.0.ffn.lin1.weight_orig', 'distilbert.transformer.layer.0.ffn.lin2.weight_mask', 'distilbert.transformer.layer.0.ffn.lin2.weight_orig', 'distilbert.transformer.layer.1.attention.k_lin.weight_mask', 'distilbert.transformer.layer.1


Pruned 모델 테스트 결과:
입력 문장: I love using Hugging Face models!
결과: [{'label': 'NEGATIVE', 'score': 0.5189980864524841}]

입력 문장: This is a terrible product.
결과: [{'label': 'NEGATIVE', 'score': 0.5235587954521179}]

입력 문장: The movie was okay, not great but not bad either.
결과: [{'label': 'NEGATIVE', 'score': 0.5224853157997131}]



In [30]:
# 입력 문장 테스트
test_sentence = "I love Hugging Face!"
result = classifier(test_sentence)
print("\n테스트 결과:", result)


테스트 결과: [{'label': 'NEGATIVE', 'score': 0.5180232524871826}]


In [31]:
import torch
from transformers import AutoModelForSequenceClassification

# 모델 경로
original_model_path = "distilbert-base-uncased-finetuned-sst-2-english"  # 원래 모델
pruned_model_path = "./pruned_sentiment_model"  # Pruning된 모델

# 1. 모델 로드
original_model = AutoModelForSequenceClassification.from_pretrained(original_model_path)
pruned_model = AutoModelForSequenceClassification.from_pretrained(pruned_model_path)

# 2. 파라미터 수 계산 함수
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())  # 전체 파라미터 수
    zero_params = sum((p == 0).sum().item() for p in model.parameters() if p.requires_grad)  # 0인 파라미터 수
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)  # 학습 가능한 파라미터 수
    return total_params, zero_params, trainable_params

# 3. 원래 모델 파라미터 수 계산
original_total, original_zero, original_trainable = count_parameters(original_model)
print(f"원래 모델 파라미터 수:")
print(f" - 전체 파라미터: {original_total}")
print(f" - 0인 파라미터: {original_zero}")
print(f" - 학습 가능한 파라미터: {original_trainable}")

# 4. Pruned 모델 파라미터 수 계산
pruned_total, pruned_zero, pruned_trainable = count_parameters(pruned_model)
print(f"\nPruned 모델 파라미터 수:")
print(f" - 전체 파라미터: {pruned_total}")
print(f" - 0인 파라미터: {pruned_zero}")
print(f" - 학습 가능한 파라미터: {pruned_trainable}")

# 5. Pruning 효과 비교
pruned_ratio = pruned_zero / pruned_total * 100
print(f"\nPruning 효과: 전체 파라미터의 {pruned_ratio:.2f}%가 0으로 설정되었습니다.")


Some weights of the model checkpoint at ./pruned_sentiment_model were not used when initializing DistilBertForSequenceClassification: ['classifier.weight_mask', 'classifier.weight_orig', 'distilbert.transformer.layer.0.attention.k_lin.weight_mask', 'distilbert.transformer.layer.0.attention.k_lin.weight_orig', 'distilbert.transformer.layer.0.attention.out_lin.weight_mask', 'distilbert.transformer.layer.0.attention.out_lin.weight_orig', 'distilbert.transformer.layer.0.attention.q_lin.weight_mask', 'distilbert.transformer.layer.0.attention.q_lin.weight_orig', 'distilbert.transformer.layer.0.attention.v_lin.weight_mask', 'distilbert.transformer.layer.0.attention.v_lin.weight_orig', 'distilbert.transformer.layer.0.ffn.lin1.weight_mask', 'distilbert.transformer.layer.0.ffn.lin1.weight_orig', 'distilbert.transformer.layer.0.ffn.lin2.weight_mask', 'distilbert.transformer.layer.0.ffn.lin2.weight_orig', 'distilbert.transformer.layer.1.attention.k_lin.weight_mask', 'distilbert.transformer.layer.1

원래 모델 파라미터 수:
 - 전체 파라미터: 66955010
 - 0인 파라미터: 0
 - 학습 가능한 파라미터: 66955010

Pruned 모델 파라미터 수:
 - 전체 파라미터: 66955010
 - 0인 파라미터: 5
 - 학습 가능한 파라미터: 66955010

Pruning 효과: 전체 파라미터의 0.00%가 0으로 설정되었습니다.


In [32]:
import torch

def remove_pruned_weights(model):
    for name, module in model.named_modules():
        if hasattr(module, "weight_orig"):
            # Pruning된 가중치를 제거하고 새로운 Dense 가중치로 교체
            module.weight = torch.nn.Parameter(module.weight.detach())
            del module.weight_orig
            del module.weight_mask
    return model

# Pruned 모델에서 불필요한 파라미터 제거
pruned_dense_model = remove_pruned_weights(pruned_model)

# 파라미터 수 다시 확인
total_params = sum(p.numel() for p in pruned_dense_model.parameters())
print(f"Pruned 모델에서 불필요한 파라미터 제거 후 총 파라미터 수: {total_params}")


Pruned 모델에서 불필요한 파라미터 제거 후 총 파라미터 수: 66955010
